In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn import datasets, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
import sklearn
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [2]:
f = pd.read_csv('/is/prices/tg_join_lab.csv', sep='|')
f['obs'] = 1
f.head()

,ticker,date,open,high,low,close,vol,price_init,volatility_5,volatility_20,...,label_50_2.0,label_50_3.0,label_50_4.0,label_50_5.0,label_50_6.0,label_50_7.0,label_50_8.0,label_50_9.0,label_50_10.0,obs
0,afks,20171217.0,10.3175,10.4325,9.2225,9.6500,91095000.0,0.0,0.455998,0.690630,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
1,afks,20180108.0,12.4625,12.6300,12.3625,12.5525,20899650.0,0.0,0.168239,1.383718,...,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1
2,afks,20180108.0,12.4625,12.6300,12.3625,12.5525,20899650.0,0.0,0.168239,1.383718,...,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1
3,afks,20180127.0,12.2525,12.3300,11.9500,12.1525,27622050.0,0.0,0.200903,0.217449,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
4,afks,20180203.0,12.0800,12.2075,11.9125,11.9800,13819500.0,0.0,0.106134,0.248238,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [9]:
4 * 2 * 2 * 2 * 62

1984

In [ ]:
from sklearn import metrics
field = 'text_reg_spell_stop_lem'
out = pd.DataFrame()

min_dfs = [8, 16, 32, 64] 
max_dfs = [1.0]
ngram_range_maxs = [(2,2), (1,1)]
j_neuts = [0.2]
j_poss = [2.0] 
j_negs = [2.0]
days = [1, 5]

counter = 0
i = 1
t = 4

learning_rates = [0.001]
n_estimators = [200]
max_depths = [4, 5]
subsamples = [0.5]





# split
for ticker in f.ticker.unique(): #
    print(ticker)

    for j_neut in j_neuts:
        for j_pos in j_poss:
            for j_neg in j_negs:
                label_neut = f'label_{i}_{j_neut}'
                label_pos = f'label_{i}_{j_pos}'
                label_neg = f'label_{i}_{j_neg}'
                
                for day in days:
                    for ngram_range_max in ngram_range_maxs:
                        for min_df in min_dfs:
                            for max_df in max_dfs:
                                for learning_rate in learning_rates:
                                    for n_estimator in n_estimators:
                                        for max_depth in max_depths:
                                            for subsample in subsamples:   
                                                try:

                                                    ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                                                    ps = ps.loc[:,~ps.columns.duplicated()]
                                                    if t == 0:
                                                        min_shape_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                                                     ps[(ps[label_neg]==-1)].shape[0])

                                                        ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                                                        ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                                                        df = pd.concat([ps1[:min_shape_df].rename(columns={label_pos:'sentiment'}), \
                                                                        ps2[:min_shape_df].rename(columns={label_neg:'sentiment'}), \
                                                                       ]).sample(frac=1, random_state=44)

                                                    else:
                                                        min_shape_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                                                     ps[(ps[label_neut]==0)].shape[0], \
                                                                     ps[(ps[label_neg]==-1)].shape[0])

                                                        ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                                                        ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                                                        ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                                                        df = pd.concat([ps1[:min_shape_df].rename(columns={label_pos:'sentiment'}), \
                                                                        ps2[:min_shape_df].rename(columns={label_neut:'sentiment'}), \
                                                                        ps3[:min_shape_df].rename(columns={label_neg:'sentiment'}) \
                                                                       ]).sample(frac=1, random_state=44)


                                                    x_train = df[field]
                                                    y_train = df['sentiment'] 

                                                # fit

                                                    tf_idf = TfidfVectorizer(min_df=min_df, \
                                                                             max_df=max_df, \
                                                                             ngram_range=ngram_range_max)

                                                    train_features = tf_idf.fit_transform(x_train)


                                                    model = XGBClassifier(random_state=42,\
                                                          learning_rate=learning_rate, \
                                                          n_estimators=n_estimator, \
                                                          max_depth=max_depth, \
                                                          subsample=subsample)
                                                    model.fit(train_features, y_train)
                                                    y_train_pred = model.predict(train_features)
                    # В идеале волатильность взять за эти же лаги наверное
                                                    df_temp = pd.DataFrame(f[(f.ticker==ticker)] \
                                                                           [['text_reg_spell_stop_lem', 'obs', 'date',\
                    #'volatility_5', 'volatility_20', 'volatility_120', 'volatility_240', 'open', 'high', 'low', 'close', 'vol', \
                                                                             f'change_close_{day}']]).\
                    reset_index().drop('index', axis=1)

                                                    y_pred = model.predict(tf_idf.transform(df_temp.text_reg_spell_stop_lem))

                                                    df_temp['sentiment'] = y_pred

                                                    df_temp['neg'] = 0
                                                    df_temp['neg'][df_temp.sentiment==-1] = 1

                                                    df_temp['pos'] = 0
                                                    df_temp['pos'][df_temp.sentiment==1] = 1

                                                    df_temp['neut'] = 0
                                                    df_temp['neut'][df_temp.sentiment==0] = 1

                                                    df_temp = df_temp.drop_duplicates().\
                                                    groupby(['date', f'change_close_{day}']).sum()

                    # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
                    # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                                    df_temp = df_temp.sort_values('date')

                                                    df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                                    df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                                    df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                                    df_temp.dropna(subset=['neg', 'pos', 'neut'], inplace=True)
                                    # просто сумма
                                                    df_temp['sum_sentiment'] = 0
                                                    df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                                    df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

                                    # отношение положительных ко всем
                                                    #df_temp['relativ_pos'] = df_temp['pos'] / (df_temp['pos'] + df_temp['neg'] + df_temp['neut'])

                                    # отношение отрицательных ко всем    
                                                    #df_temp['relativ_neg'] = df_temp['neg'] / (df_temp['pos'] + df_temp['neg'] + df_temp['neut'])
                                    # дивергенция c добавлением 
                                                    df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                                              / (df_temp.pos + df_temp.neg + df_temp.neut)**2)**(1/2)

                                                    df_temp.reset_index(level=[f'change_close_{day}', 'date'], inplace=True)
                                                    df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                                    df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                                    res = dict()
                                                    res['counter'] = [counter]
                                                    res['ticker'] = [ticker]
                                                    res['date'] = [day]

                                                    res['min_df'] = [min_df]
                                                    res['max_df'] = [max_df]

                                                    res['j_neut'] = [j_neut]
                                                    res['j_pos'] = [j_pos] 
                                                    res['j_neg'] = [j_neg] 

                                                    res['ngram_range_max'] = [ngram_range_max]
                                                    res['learning_rate'] = [learning_rate]
                                                    res['n_estimator'] = [n_estimator]
                                                    res['max_depth'] = [max_depth]
                                                    res['subsample'] = [subsample]

                                                    res['train_size'] = [y_train.shape[0]]
                                                    res['test_size'] = [y_pred.shape[0]]

                                                    res['obs'] = \
                                                    [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]

                                                    res['original_return_isna_all'] = \
                                                    [(df_temp[f'change_close_{day}']\
                                                      [df_temp[f'change_close_{day}'].isna()==False]).sum()]

                                                    res['model_return_isna_all'] = \
                                                    [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                                    res['model_return_isna_short'] = \
                                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                                    res['model_return_isna_long'] = \
                                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                                 (df_temp.sum_sentiment!=-1)]).sum()]


                                                    res['model_diver_return_isna_all'] = \
                                                    [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                                    res['model_diver_return_isna_short'] = \
                                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                                    res['model_diver_return_isna_long'] = \
                                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                                 (df_temp.sum_sentiment!=-1)]).sum()]






                                                    res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                                    res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                            , average='macro')]


                                                    res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                                            , average='micro')]


                                                    res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                                      , average='micro')]


                                                    res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                                      , average='macro')]

                                                    res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                              , average='micro')]


                                                    res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                                              , average='macro')]




                                                    counter += 1
                                                    print(counter)


                                                    res = pd.DataFrame.from_dict(res)
                                                    out = out.append(pd.DataFrame(res))
                                                    if counter % 100 == 0:
                                                        out.to_excel(f'/is/res/xgb_tg/xgb_ind.xlsx', index=False)
                                                except:
                                                    continue
                                

                    


out.to_excel(f'/is/res/xgb_tg/xgb_ind.xlsx', index=False)

afks
[12:14:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1
[12:14:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2
[12:15:11] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
3
[12:15:37] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to res

[12:42:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
33
[12:42:29] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
34
[12:42:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
35
[12:43:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to resto

[13:10:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
65
[13:11:31] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
70
[13:13:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
71
[13:14:17] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to resto

100
[13:49:35] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
101
[13:50:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
102
[13:50:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
103
[13:50:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like t

132
[14:25:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
133
[14:26:12] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
134
[14:26:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
135
[14:26:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like t

...

1828
[13:32:31] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1829
[13:32:43] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1830
[13:32:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1831
[13:33:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd li

1860
[13:51:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1861
[13:51:45] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1862
[13:51:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1863
[13:52:10] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd li

1892
[14:10:53] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1893
[14:11:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1894
[14:11:21] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1895
[14:11:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd li

1924
[14:31:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1925
[14:31:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1926
[14:31:44] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1927
[14:31:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd li

1956
[14:51:21] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1957
[14:51:35] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1958
[14:51:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1959
[14:52:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd li

In [ ]:
out.describe()